<a href="https://colab.research.google.com/github/CalculatedContent/fine-tuning-demo/blob/main/WWFineTuneAlbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WW Fine Tune Albert

In [1]:
from getpass import getpass
token = getpass('Enter your GitHub token: ')

Enter your GitHub token: ··········


In [ ]:
import os
os.environ["WW_GITHUB_TOKEN"] = token

In [ ]:
!which git-lfs

In [4]:
!git clone https://{token}@github.com/CalculatedContent/WW_Reg_Exp_Data.git
!git clone https://{token}@github.com/CalculatedContent/WW_Regularizer_Experiments.git
%cd /content/WW_Regularizer_Experiments


Cloning into 'WW_Reg_Exp_Data'...
remote: Enumerating objects: 162, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 162 (delta 11), reused 92 (delta 4), pack-reused 56 (from 2)
Receiving objects: 100% (162/162), 62.47 MiB | 42.85 MiB/s, done.
Resolving deltas: 100% (20/20), done.
Cloning into 'WW_Regularizer_Experiments'...
remote: Enumerating objects: 725, done.
remote: Counting objects: 100% (290/290), done.
remote: Compressing objects: 100% (203/203), done.
Receiving objects: 100% (725/725), 2.15 MiB | 25.85 MiB/s, done.
remote: Total 725 (delta 175), reused 87 (delta 87), pack-reused 435 (from 2)
Resolving deltas: 100% (413/413), done.
/content/WW_Regularizer_Experiments


In [5]:
!git pull

Already up to date.


In [6]:
!pip install torch.optim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.0/136.0 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.8/343.8 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.2/70.2 kB 6.6 MB/s eta 0:00:00


In [7]:
import os, pathlib

import sys
sys.path.append("/content/WW_Regularizer_Experiments")

# Choose estimator: soft_mle | hill | quantile
os.environ["WW_ESTIMATOR"] = "soft_mle"

# Model & outputs
os.environ["MODEL_ID"]    = "prajjwal1/bert-tiny"
os.environ["OUTPUT_DIR"]  = "/content/ww_outputs"

# Mirror artifacts to Drive (optional)
drive_dir = "/content/drive/MyDrive/wwruns/run_demo"
pathlib.Path(drive_dir).mkdir(parents=True, exist_ok=True)
os.environ["DRIVE_DIR"]   = drive_dir

# Leave blank for time-based seed, or set to e.g. "42"
os.environ["WW_SEED"]     = ""


In [8]:
!pip install -q weightwatcher transformers datasets
#muon-optimizer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 5.1 MB/s eta 0:00:00


In [9]:
!pip install -e .


Obtaining file:///content/WW_Regularizer_Experiments
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ww-regularizer-experiments (pyproject.toml) ... done
  Created wheel for ww-regularizer-experiments: filename=ww_regularizer_experiments-0.1.0-0.editable-py3-none-any.whl size=11306 sha256=911358307321a9e0d17b15bf1a3181f8e987674d969b46a2b41722101debfbd4
  Stored in directory: /tmp/pip-ephem-wheel-cache-mpeeowc5/wheels/f3/89/b9/42f42e018a623fa453c25f98f5ea231d150a66fa3a9f225a14
Successfully built ww-regularizer-experiments


# ALBERT

In [ ]:
# Assumes you already ran:
# from getpass import getpass
# token = getpass('Enter your GitHub token: ')

# --- Train ALBERT on GLUE MRPC for 10 epochs with multiple optimizers, save artifacts, and push to GitHub ---

import os

# 1) Runtime/env config
os.environ["MODEL_ID"]   = "albert-base-v2"    # << ALBERT
os.environ["OUTPUT_DIR"] = "outputs_albert_mrpc"
os.environ["WW_ESTIMATOR"] = "none"            # ignored for baselines
os.environ["WW_SEED"] = ""                     # blank => time-based seed
os.environ.setdefault("WW_DEBUG_MUON", "0")

# (Optional) pass your PAT to our publisher via env so it can set the HTTPS remote for pushing
os.environ["GITHUB_TOKEN"] = os.environ.get("GITHUB_TOKEN", "") or "___REPLACE_ME_IF_NOT_USING_token___" if 'token' not in globals() else token

# 2) Imports
from wwreg.core import HP_PRESETS
from wwreg.datasets import load_glue_mrpc
from wwreg.runner import ExperimentRunner
from wwreg.utils.publish import publish_run_to_git
from wwreg.plotting import plot_results

# 3) Hyperparameters (10 epochs for every optimizer)
EPOCHS = 10
HP_PRESETS["AdamW"].epochs       = EPOCHS
HP_PRESETS["AdamW"].weight_decay = 0.05
HP_PRESETS["Adam"].epochs        = EPOCHS
HP_PRESETS["Adam"].weight_decay  = 1e-3
HP_PRESETS["SGD"].epochs         = max(EPOCHS, HP_PRESETS["SGD"].epochs)
HP_PRESETS["Muon"].epochs        = EPOCHS
HP_PRESETS["Muon"].lr            = 2e-5
HP_PRESETS["Muon"].weight_decay  = 5e-2
HP_PRESETS["Muon"].warmup_ratio  = 0.20
HP_PRESETS["Muon"].batch_size    = 16

# 4) Launch training (with WW diagnostics; disable alpha-based early stop)
runner = ExperimentRunner(
    model_id=os.environ["MODEL_ID"],
    num_labels=2,
    dataset_loader=load_glue_mrpc,            # reuse built-in MRPC loader
    output_dir=os.environ["OUTPUT_DIR"],
    layer_dropout=None
)

paths = runner.run(
    variants=["AdamW", "Adam", "SGD", "Muon"],
    ww_reg=None,                              # no WW regularizer (pure baselines)
    hp_presets=HP_PRESETS,
    seed=None,
    per_run_seeds=[None, None, None, None],
    dataset_kwargs={"max_len": 128},
    warmup_optimizer="AdamW",                 # 1-epoch warmup happens inside runner
    enable_weightwatcher=True,                # keep WW alpha/xmin per epoch
    ww_options=dict(savefig=False, plot=False),
    estimator_name="baseline",
    halt_on_low_delta_alpha=False             # << do NOT stop early on low alpha
)

print("✅ Finished ALBERT (MRPC) baselines")
print("Artifacts:", paths)
print("All outputs saved in:", os.path.abspath(os.environ["OUTPUT_DIR"]))

# 5) (Optional) Quick alpha-per-layer/per-epoch plots using built-in helper
#    This will produce one figure per layer per optimizer (Alpha + Tail plots).
plot_results(
    sources="outputs_albert_mrpc",
    variants=["AdamW", "Adam", "SGD", "Muon"],
    metrics=["eval_accuracy", "eval_f1", "eval_loss",  "train_accuracy", "train_f1", "train_loss"],  # ← include accuracy + loss
    layerwise_per_optimizer=True,   # one α-vs-epoch plot per optimizer
    show=True                       # True = display inline, False = save only
);


# 6) Publish artifacts to your GitHub repo under ./outputs/outputs_albert_mrpc/
#    Default exclude_globs in publish_run_to_git skips big checkpoint files (*.bin, *.pt, checkpoint-*)
#    so only .feather/.csv/.json and optional .png plots are committed.
publish_res = publish_run_to_git(
    run_dir=os.environ["OUTPUT_DIR"],
    repo_path="/content/WW_Reg_Exp_Data",
    repo_url=f"https://{os.environ.get('GITHUB_TOKEN','')}@github.com/CalculatedContent/WW_Reg_Exp_Data.git",
    dest_subdir="outputs",
    push=True,
    fail_if_exists=False,   # overwrite prior run folder if it exists
)
print("✅ Committed:", publish_res.commit_hash)
print("📂 Saved to repo:", publish_res.run_dest_path)

# (Optional) show last commit in the local repo so you can confirm the push worked
try:
    import subprocess, shlex
    repo_path = publish_res.repo_path.as_posix()
    print("\nLast commit in local repo:")
    print(subprocess.check_output(shlex.split(f"git -C {repo_path} --no-pager log -1 --oneline")).decode().strip())
    print("\nTree at destination folder:")
    print(subprocess.check_output(shlex.split(f"ls -la {shlex.quote(publish_res.run_path if hasattr(publish_res,'run_path') else publish_res.run_dest_path.as_posix())}")).decode())
except Exception as e:
    print("Note: Skipping local git log/ls check:", e)

experiment_id=20251027-205943-777604e5
[WW] No seed provided — using current time as seed: 1761598783


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

mrpc/train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

mrpc/validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

...

[WW][epoch 1] AdamW: train_loss=0.3746 train_acc=nan train_f1=nan | eval_loss=0.3827 eval_acc=0.8309 eval_f1=0.8852
[WW][epoch 1] AdamW layer Embedding (id 3): WW α=3.078 tail≈39.8%
[WW][epoch 1] AdamW layer Embedding (id 4): WW α=2.430 tail≈6.2%
[WW][epoch 1] AdamW layer Linear (id 9): WW α=1.749 tail≈31.2%
[WW][epoch 1] AdamW layer Linear (id 16): WW α=2.069 tail≈5.9%
[WW][epoch 1] AdamW layer Linear (id 17): WW α=2.050 tail≈7.2%
[WW][epoch 1] AdamW layer Linear (id 18): WW α=1.719 tail≈22.0%
[WW][epoch 1] AdamW layer Linear (id 21): WW α=1.770 tail≈14.6%
[WW][epoch 1] AdamW layer Linear (id 23): WW α=2.059 tail≈8.2%
[WW][epoch 1] AdamW layer Linear (id 24): WW α=1.806 tail≈18.5%
[WW][epoch 1] AdamW layer Linear (id 27): WW α=1.681 tail≈6.1%
[WW][dropout] epoch 1 AdamW: applied p=0.050 to albert.encoder.albert_layer_groups.0.albert_layers.0.attention.key, albert.encoder.albert_layer_groups.0.albert_layers.0.ffn, albert.encoder.albert_layer_groups.0.albert_layers.0.ffn_output, classif

[WW][epoch 1] Adam: train_loss=0.3337 train_acc=nan train_f1=nan | eval_loss=0.3345 eval_acc=0.8652 eval_f1=0.8995
[WW][epoch 1] Adam layer Embedding (id 3): WW α=4.466 tail≈55.5%
[WW][epoch 1] Adam layer Embedding (id 4): WW α=1.770 tail≈71.9%
[WW][epoch 1] Adam layer Linear (id 9): WW α=2.009 tail≈57.8%
[WW][epoch 1] Adam layer Linear (id 16): WW α=4.058 tail≈6.2%
[WW][epoch 1] Adam layer Linear (id 17): WW α=4.248 tail≈5.5%
[WW][epoch 1] Adam layer Linear (id 18): WW α=4.113 tail≈5.6%
[WW][epoch 1] Adam layer Linear (id 21): WW α=3.819 tail≈4.0%
[WW][epoch 1] Adam layer Linear (id 23): WW α=4.683 tail≈6.4%
[WW][epoch 1] Adam layer Linear (id 24): WW α=3.221 tail≈14.5%
[WW][epoch 1] Adam layer Linear (id 27): WW α=2.553 tail≈10.7%
[WW][dropout] epoch 1 Adam: applied p=0.050 to albert.encoder.albert_layer_groups.0.albert_layers.0.attention.query
[WW][epoch 1] Adam: WW α≈3.494; tail≈23.8%
[WW][epoch 2] Adam: train_loss=0.2492 train_acc=nan train_f1=nan | eval_loss=0.3191 eval_acc=0.872

[WW][epoch 1] SGD: train_loss=0.4229 train_acc=nan train_f1=nan | eval_loss=0.4198 eval_acc=0.8064 eval_f1=0.8686
[WW][epoch 1] SGD layer Embedding (id 3): WW α=3.100 tail≈51.6%
[WW][epoch 1] SGD layer Embedding (id 4): WW α=2.172 tail≈4.7%
[WW][epoch 1] SGD layer Linear (id 9): WW α=1.809 tail≈21.1%
[WW][epoch 1] SGD layer Linear (id 16): WW α=1.973 tail≈9.9%
[WW][epoch 1] SGD layer Linear (id 17): WW α=1.859 tail≈15.2%
[WW][epoch 1] SGD layer Linear (id 18): WW α=1.828 tail≈12.0%
[WW][epoch 1] SGD layer Linear (id 21): WW α=1.784 tail≈13.9%
[WW][epoch 1] SGD layer Linear (id 23): WW α=2.027 tail≈10.7%
[WW][epoch 1] SGD layer Linear (id 24): WW α=1.792 tail≈23.8%
[WW][epoch 1] SGD layer Linear (id 27): WW α=1.678 tail≈4.2%
[WW][dropout] epoch 1 SGD: applied p=0.050 to albert.encoder.albert_layer_groups.0.albert_layers.0.attention.dense, albert.encoder.albert_layer_groups.0.albert_layers.0.attention.key, albert.encoder.albert_layer_groups.0.albert_layers.0.attention.value, albert.encod

[WW][epoch 1] Muon: train_loss=0.4617 train_acc=nan train_f1=nan | eval_loss=0.4780 eval_acc=0.7770 eval_f1=0.8501
[WW][epoch 1] Muon layer Embedding (id 3): WW α=3.694 tail≈50.0%
[WW][epoch 1] Muon layer Embedding (id 4): WW α=2.713 tail≈3.1%
[WW][epoch 1] Muon layer Linear (id 9): WW α=1.887 tail≈24.2%
[WW][epoch 1] Muon layer Linear (id 16): WW α=2.003 tail≈9.5%
[WW][epoch 1] Muon layer Linear (id 17): WW α=1.878 tail≈14.7%
[WW][epoch 1] Muon layer Linear (id 18): WW α=1.851 tail≈11.7%
[WW][epoch 1] Muon layer Linear (id 21): WW α=1.807 tail≈14.1%
[WW][epoch 1] Muon layer Linear (id 23): WW α=2.055 tail≈10.0%
[WW][epoch 1] Muon layer Linear (id 24): WW α=1.807 tail≈24.2%
[WW][epoch 1] Muon layer Linear (id 27): WW α=1.691 tail≈4.2%
[WW][dropout] epoch 1 Muon: applied p=0.050 to albert.encoder.albert_layer_groups.0.albert_layers.0.attention.dense, albert.encoder.albert_layer_groups.0.albert_layers.0.attention.key, albert.encoder.albert_layer_groups.0.albert_layers.0.ffn, albert.encod

[WW][epoch 5] Muon: train_loss=0.5997 train_acc=nan train_f1=nan | eval_loss=0.6297 eval_acc=0.7255 eval_f1=0.8255


# 5 RESTARTS FOR ERRORBARS

In [ ]:
# --- Train ALBERT (MRPC) baselines for 10 epochs, run 5 replicates for error bars, save & push to GitHub ---

import os, time, random

# 1) Runtime/env config
os.environ["MODEL_ID"]     = "albert-base-v2"
os.environ["OUTPUT_DIR"]   = "outputs_albert_mrpc"
os.environ["WW_ESTIMATOR"] = "none"
os.environ["WW_SEED"]      = ""                      # blank => time-based seed
os.environ.setdefault("WW_DEBUG_MUON", "0")

# Optional: GitHub token for publisher
os.environ["GITHUB_TOKEN"] = os.environ.get("GITHUB_TOKEN", "") or ("___REPLACE_ME_IF_NOT_USING_token___" if 'token' not in globals() else token)

# 2) Imports
from wwreg.core import HP_PRESETS
from wwreg.datasets import load_glue_mrpc
from wwreg.runner import ExperimentRunner
from wwreg.utils.publish import publish_run_to_git
from wwreg.plotting import plot_results

# 3) Hyperparameters (10 epochs for every optimizer)
EPOCHS = 10
HP_PRESETS["AdamW"].epochs       = EPOCHS; HP_PRESETS["AdamW"].weight_decay = 0.05
HP_PRESETS["Adam"].epochs        = EPOCHS; HP_PRESETS["Adam"].weight_decay  = 1e-3
HP_PRESETS["SGD"].epochs         = max(EPOCHS, HP_PRESETS["SGD"].epochs)
HP_PRESETS["Muon"].epochs        = EPOCHS
HP_PRESETS["Muon"].lr            = 2e-5; HP_PRESETS["Muon"].weight_decay = 5e-2
HP_PRESETS["Muon"].warmup_ratio  = 0.20; HP_PRESETS["Muon"].batch_size = 16

optim_variants = ["AdamW", "Adam", "SGD", "Muon"]
N_REPS = 5

# 4) Run 5 replicates (experiment_number=1..5) — accumulate to same output folder
for rep in range(1, N_REPS + 1):
    base_seed = int(time.time()) ^ (rep * 7919) ^ random.getrandbits(31)

    runner = ExperimentRunner(
        model_id=os.environ["MODEL_ID"],
        num_labels=2,
        dataset_loader=load_glue_mrpc,
        output_dir=os.environ["OUTPUT_DIR"],
        layer_dropout=None,
    )

    _paths = runner.run(
        variants=optim_variants,
        ww_reg=None,
        hp_presets=HP_PRESETS,
        seed=base_seed,
        per_run_seeds=[base_seed + i for i in range(len(optim_variants))],
        dataset_kwargs={"max_len": 128},
        warmup_optimizer="AdamW",
        enable_weightwatcher=True,
        ww_options=dict(savefig=False, plot=False),
        estimator_name="baseline",
        experiment_number=rep,             # <-- allows aggregation for error bars
        halt_on_low_delta_alpha=False,
    )
    print(f"✅ Finished replicate {rep}: {_paths.get('merged_feather','(no merged)')}")

print("✅ All 5 replicates complete. Outputs in:", os.path.abspath(os.environ["OUTPUT_DIR"]))

# 5) Plot metrics with error bars across the 5 replicates
plot_results(
    sources=os.environ["OUTPUT_DIR"],
    variants=optim_variants,
    metrics=["eval_accuracy", "eval_f1", "eval_loss"],
    errorbars=True,            # enable shaded bands / yerr
    error="std",               # "std" | "sem" | "ci95"
    show=True,
)

# Optional: α vs epoch plots per optimizer
plot_results(
    sources=os.environ["OUTPUT_DIR"],
    variants=optim_variants,
    layerwise_per_optimizer=True,
    show=True,
)

# 6) Publish artifacts to GitHub
publish_res = publish_run_to_git(
    run_dir=os.environ["OUTPUT_DIR"],
    repo_path="/content/WW_Reg_Exp_Data",
    repo_url=f"https://{os.environ.get('GITHUB_TOKEN','')}@github.com/CalculatedContent/WW_Reg_Exp_Data.git",
    dest_subdir="outputs",
    push=True,
    fail_if_exists=False,
)
print("✅ Committed:", publish_res.commit_hash)
print("📂 Saved to repo:", publish_res.run_dest_path)

experiment_id=20251027-212927-72aa5e8e


Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/content/WW_Regularizer_Experiments/wwreg/runner.py:1021: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  warm_trainer = Trainer(


Step,Training Loss
50,0.618000
100,0.558500


[WW] AdamW: max_grad_norm=1.0
[WW] Trainer args for AdamW: lr=2e-05, weight_decay=0.05, max_grad_norm=1.0


Step,Training Loss
50,0.518200


Step,Training Loss
50,0.518200
